In [1]:
import pandas as pd
import numpy as np
import re,pyperclip

In [2]:
repL4L="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/"
fSwim=repL4L+"IMM21-00-X-paradigmes-Swim1.csv"
fSwim=repL4L+"IMM21-00-X-paradigmes-Swim2.csv"
fSwim=repL4L+"IMM21-00-X-Morphomes-Swim1.csv"
fSwim=repL4L+"IMM21-00-X-Morphomes-Swim2.csv"
fControl=repL4L+"IMM21-02-X-paradigmes.csv"

In [3]:
dfSwim=pd.read_csv(fSwim,sep=";",encoding="utf8")
dfControl=pd.read_csv(fControl,sep=";",encoding="utf8")

In [4]:
cols=dfControl.columns.tolist()[2:]
# cols

In [5]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
neutralisations=neutralisationsSUD

bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [8]:
def checkFrench(prononciation):
    if prononciation and not pd.isnull(prononciation):
        result=recoder(prononciation)
        m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
        if m:
            print ("pb avec un glide final", [prononciation])
        m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
        if m:
            n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
            if not n:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
        m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
        if m:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
    else:
        result=prononciation
    return result

In [9]:
equals={}
differents={}
missings={}

def normalise(forme):
    result=recoder(forme)
    result=checkFrench(result)
    return result

def compare(row):
    lexeme=row.lexeme
    if lexeme in dfSwim["lexeme"].tolist():
        lEquals=equals[lexeme]={}
        lDifferents=differents[lexeme]={}
        lMissings=missings[lexeme]={}
        swimRow=dfSwim[dfSwim.lexeme.str.contains("^"+lexeme+"$")]
        for c in cols:
            swimRowC=swimRow[c].values[0]
            if row[c]==row[c]:
                if swimRowC==swimRowC:
                    if normalise(row[c])==normalise(swimRowC):
                        lEquals[c]=(row[c],swimRowC)
                    else:
                        lDifferents[c]=(row[c],swimRowC)
                else:
                    lMissings[c]=(row[c],swimRowC)
        return (lexeme,len(lEquals),len(lDifferents),len(lMissings))
    return (lexeme,np.nan,np.nan,np.nan)

In [10]:
dfResults=pd.DataFrame([[l,e,d,m] for l,e,d,m in dfControl.apply(compare,axis=1).values], columns="lexeme equal different missing".split(" "))

In [11]:
nbForms=dfResults["equal different missing".split(" ")].sum().sum()
nbForms

45042.0

In [12]:
print fSwim.split("/")[-1]
# Paradigmes-Swim1: 43
# Paradigmes-Swim2: 92
# Morphomes-Swim1: 42
# Morphomes-Swim2: 92
ajustementsManuels=42
nbEqual=int(dfResults["equal"].sum()+ajustementsManuels)
nbDifferent=int(dfResults["different"].sum()-ajustementsManuels)
nbMissing=int(dfResults["missing"].sum())
print "formes identiques :",nbEqual,"=> %2.1f%%"%round(nbEqual/nbForms*100,1)
print "formes manquantes :",nbMissing,"=> %2.1f%%"%round(nbMissing/nbForms*100,1)
print "formes différentes :",nbDifferent,"=> %2.1f%%"%round(nbDifferent/nbForms*100,1)


IMM21-00-X-Morphomes-Swim2.csv
formes identiques : 43142 => 95.8%
formes manquantes : 1731 => 3.8%
formes différentes : 169 => 0.4%


In [13]:
dfResults.count()

lexeme       6368
equal        4746
different    4746
missing      4746
dtype: int64

In [14]:
tMissings=[]
for k,v in missings.iteritems():
    for kk,vv in v.iteritems():
        tMissings.append("\t".join([k,kk,vv[0]]))
pyperclip.copy("\n".join(tMissings))

In [15]:
dfDiff=pd.DataFrame.from_dict(differents).T

In [16]:
for k,v in differents.iteritems():
    for kk,vv in v.iteritems():
        print "\t".join([k,kk,vv[0],vv[1]])
        

débrayer	pi3P	dEbrE	dEbrE,dEbrEj
répugner	ii2P	rEpyJe	rEpyJje
reclassifier	pi1P	r6klasifjô	r6klasifisô
reclassifier	pi3P	r6klasifi	r6klasifis
reclassifier	ppFS	r6klasifje	r6klasifi
contresigner	ii2P	kôtr6siJe	kôtr6siJje
aligner	ii2P	aliJe	aliJje
aligner	ps2P	aliJe	aliJje
sous-payer	pi1S	supE	supEj
sous-payer	pi3P	supE	supEj
sous-payer	pi3S	supE	supEj
nantir	ii2P	nâtisje	nâtje
rayer	pi1S	rE	rE,rEj
rayer	pI2S	rE	rE,rEj
rayer	pi3P	rE	rE,rEj
assigner	ii2P	asiJe	asiJje
imprégner	ii2P	êprEJe	êprEJje
embarrer	inf	âbae	âbaer
embarrer	pi2S	âba	âbae
cosigner	ii2P	kOsiJe	kOsiJje
décoller	is1P	dEkOlasjô	dEkOle
prévoir	ai3P	prEvir	prEvyr
valiser	ai3S	vliz	vliza
désennuyer	fi3S	dEzânHira	dEzânHij6ra
désennuyer	pi3P	dEzânHi	dEzânHij
désennuyer	pi3S	dEzânHi	dEzânHij
souligner	ii2P	suliJe	suliJje
admirer	is1P	admirasjô	admire
rougeoyer	pi3P	ruZwa	ruZwaj
rougeoyer	pi2S	ruZwa	ruZwaj
rougeoyer	pi3S	ruZwa	ruZwaj
grogner	ii2P	grOJe	grOJje
friter	ii1S	frit	fritE
friter	ii3P	frit	fritE
pagayer	pi1S	pagE	pagE,